In [1]:
import joblib as jl
import soundfile as sf
import torch
from tqdm.auto import tqdm

from diff_ttsg.hifigan.env import AttrDict
from diff_ttsg.hifigan.models import Generator as HifiGAN
from diff_ttsg.models.diff_ttsg import Diff_TTSG
from diff_ttsg.utils.model import denormalize
from diff_ttsg.utils.utils import intersperse
from pymo.writers import BVHWriter


from diff_ttsg.text import text_to_sequence, cmudict, sequence_to_text


In [2]:
DIFF_TTSG_CHECKPOINT = "logs/train/dev/runs/2023-06-21_15-52-26/checkpoints/epoch_epoch=001.ckpt"
HIFIGAN_CHECKPOINT = "g_02500000"
MOTION_PIPELINE = "diff_ttsg/resources/data_pipe.expmap_86.1328125fps.sav"
CMU_DICT_PATH = "diff_ttsg/resources/cmu_dictionary"

In [5]:
def load_model(checkpoint_path):
    model = Diff_TTSG.load_from_checkpoint(checkpoint_path)
    model.eval()
    return model
model = load_model(DIFF_TTSG_CHECKPOINT)

In [8]:
def load_vocoder(checkpoint_path):
    

<bound method Diff_TTSG.synthesise of Diff_TTSG(
  (encoder): TextEncoder(
    (emb): Embedding(149, 192)
    (prenet): ConvReluNorm(
      (conv_layers): ModuleList(
        (0-2): 3 x Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (norm_layers): ModuleList(
        (0-2): 3 x LayerNorm()
      )
      (relu_drop): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.5, inplace=False)
      )
      (proj): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (encoder): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0-5): 6 x MultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_1): M